# Part 3. Multi Level Perceptron from Scratch 

In the previous section, we attempted to train a simple 2 layer MLP on Keras. Keras, being a high level abstracted framework, hides the details behind the model and simplifies the process. We will now try to build our own 2 layer MLP, purely out of NumPy, which will unveil the hidden components of neural network training. Similar to past from-scratch attempts, we will start by creating a class.

## 1. Create a class `MLPTwoLayers`

- One of the starting points to take care of while building your network is to initialize your weight matrix correctly. Consider appropriate sizes for your input, hidden and output layers - your __init__ method should take in the params `input_size`, `hidden_size`, `output_size`. Then, using these variables, initialise the weights for the hidden layers `w1`, `w2`, `b1`, and `b2`.

In [1]:
%load_ext autoreload
%autoreload 2
from src.mlp import MLPTwoLayers as MLP

ModuleNotFoundError: No module named 'src.mlp'

In [ ]:
mlp = MLP(3072, 100, 10)

## 2. Create a `forward ` method, which takes in a set of features
- Create the `forward` method to calculate the predicted class probabilities of an image. This is known as a forward pass.  You should wrap the hidden layer with a sigmoid function (or others if you prefer), and the output layer with a softmax function.

In [ ]:
# import your data preparation methods here, ensure your data is randomized
preds = mlp.forward(X[0])
preds

## 3. Create a `loss` method, which takes in the predicted probability and actual label
- Compute the loss function: This is a function of the actual label y and predicted label y. It captures how far off our predictions are from the actual target. The objective is to minimize this loss function. 

In [ ]:
train_loss = mlp.loss(preds, y[0])
train_loss

## 4. Create a `backward` method, which takes in the loss
- Using the backpropogation algorithm, execute the backward pass and adjust the weights and bias accordingly
- You can use a default learning rate of 1e-3 for this exercise. If you would like do otherwise, you can try to implement it as a parameter.

In [ ]:
mlp.backward(train_loss)

Now, we can try training the model.

In [ ]:
# initial attempt at training
test_loss = 0
for i in range(3000, 3500):
    test_loss += mlp.loss(mlp.forward(X[i]), y[i])
print(test_loss / 500)

In [ ]:
for i in range(3000):
    if i % 100:
        print('Item {}'.format(i))
    mlp.backward(mlp.loss(mlp.forward(X[i]), y[i]))

Finally, re-test your model.

In [ ]:
test_loss = 0
for i in range(3000, 3500):
    test_loss += mlp.loss(mlp.forward(X[i]), y[i])
print(test_loss / 500)

Hopefully, you see that your test loss has decreased after training!

# Part 4. Convolutional Neural Network (CNN)
Please attempt this section only after you have completed the rest!

In the previous part, you implemented a multilayer perceptron network on CIFAR-10. The implementation was simple but not very modular since the loss and gradient were computed in a single monolithic function. This is manageable for a simple two-layer network, but would become impractical as you move to bigger models. Ideally, you want to build networks using a more modular design so that you can implement different layer types in isolation and then snap them together into models with different architectures.

In this part of exercise, you will implement a close to state-of-the-art deep learning model for CIFAR-10 with the Keras Deep Learning library. In addition to implementing convolutional networks of various depth, you will need to explore different update rules for optimization, and introduce **Dropout** as a regularizer, **Batch Normalization** and **Data Augmentation** as a tool to more efficiently optimize deep networks.

We saw models performing >98% accuracy on `CIFAR-10`, while most state-of-the-art models cross the 97% boundary. In general, models beyond **95%** are fairly decent.

## Reading resources

[Dropout](http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf?utm_content=buffer79b43&utm_medium=social&utm_source=twitter.com&utm_campaign=buffer) is a regularization technique for overfitting in neural networks by preventing complex co-adaptations on training data. It is a very efficient way of performing model averaging with neural networks.

[Batch Normalization](https://pdfs.semanticscholar.org/c1ba/ed41e4bc9401b1b2ec8ef55ba45543f7a1a3.pdf) is a technique to provide any layer in a neural network with inputs that are zero mean/unit variance.

[Data Augmentation](https://medium.com/nanonets/how-to-use-deep-learning-when-you-have-limited-data-part-2-data-augmentation-c26971dc8ced) means increasing the number of data points. In terms of images, it may mean that increasing the number of images in the dataset.

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from src.load_img import LoadImage
from sklearn.model_selection import train_test_split
from keras import optimizers

from keras.layers import Dense, Dropout
from keras.layers import Flatten,  MaxPooling2D, Conv2D,BatchNormalization
from keras.callbacks import EarlyStopping
import pandas as pd

Using TensorFlow backend.


In [3]:
imgloader = LoadImage()
imgloader.load_unpickledata()
train_data,train_labels,test_data,test_labels=imgloader.getVisData()

In [4]:
train_data=train_data/255
test_data=test_data/255
n_classes = 10
train_labels = keras.utils.to_categorical(train_labels, n_classes)
test_labels = keras.utils.to_categorical(test_labels, n_classes)

In [6]:
X_train,X_test,y_train,y_test= train_test_split(train_data,train_labels,test_size=0.25,random_state=42)

- Enhancing the performance of you existing model in part 2 with convolutional neural networks
- The implementation of model should be done by using Keras (or PyTorch)
- Train your designed model 
- Improve performance with algorithm tuning: Dropout, Batch normalization, Data augmentation and other optimizers

__Baseline CNN Model__

In [7]:
cnn_model_base = Sequential()
cnn_model_base.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(32,32,3)))
cnn_model_base.add(Conv2D(32, kernel_size=3, activation='relu'))
cnn_model_base.add(Flatten())
cnn_model_base.add(Dense(10, activation='softmax'))

In [8]:
cnn_model_base.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
early_stopping_monitor = EarlyStopping(patience=3)
cnn_model_base.fit(X_train, y_train,  epochs=10, verbose=1,validation_data=(X_test,y_test), callbacks=[early_stopping_monitor])

Train on 37500 samples, validate on 12500 samples
Epoch 1/10
37500/37500 [==============================] - 29s 762us/step - loss: 1.4884 - acc: 0.4705 - val_loss: 1.2789 - val_acc: 0.5487
Epoch 2/10
37500/37500 [==============================] - 25s 672us/step - loss: 1.1484 - acc: 0.5999 - val_loss: 1.1755 - val_acc: 0.5954
Epoch 3/10
37500/37500 [==============================] - 26s 681us/step - loss: 0.9790 - acc: 0.6620 - val_loss: 1.0678 - val_acc: 0.6258
Epoch 4/10
37500/37500 [==============================] - 26s 693us/step - loss: 0.8456 - acc: 0.7064 - val_loss: 1.1016 - val_acc: 0.6255
Epoch 5/10
37500/37500 [==============================] - 26s 690us/step - loss: 0.7279 - acc: 0.7484 - val_loss: 1.1282 - val_acc: 0.6257
Epoch 6/10
37500/37500 [==============================] - 26s 688us/step - loss: 0.6186 - acc: 0.7855 - val_loss: 1.2033 - val_acc: 0.6186


__CNN Model with Dropout__
- Will randomly drop neurons in every pass

In [10]:
cnn_model_dropout = Sequential()
cnn_model_dropout.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(32,32,3)))
cnn_model_dropout.add(Dropout(0.25))
cnn_model_dropout.add(Conv2D(32, kernel_size=3, activation='relu'))
cnn_model_dropout.add(Dropout(0.25))

cnn_model_dropout.add(Flatten())
cnn_model_dropout.add(Dense(10, activation='softmax'))

In [11]:
cnn_model_dropout.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
early_stopping_monitor = EarlyStopping(patience=3)
cnn_model_dropout.fit(X_train, y_train,  epochs=10, verbose=1,validation_data=(X_test,y_test), callbacks=[early_stopping_monitor])

Train on 37500 samples, validate on 12500 samples
Epoch 1/10
37500/37500 [==============================] - 38s 1ms/step - loss: 1.5395 - acc: 0.4483 - val_loss: 1.2618 - val_acc: 0.5555
Epoch 2/10
37500/37500 [==============================] - 38s 1ms/step - loss: 1.1908 - acc: 0.5807 - val_loss: 1.1530 - val_acc: 0.6007
Epoch 3/10
37500/37500 [==============================] - 38s 1ms/step - loss: 1.0555 - acc: 0.6312 - val_loss: 1.1240 - val_acc: 0.6058
Epoch 4/10
37500/37500 [==============================] - 37s 995us/step - loss: 0.9427 - acc: 0.6717 - val_loss: 1.1036 - val_acc: 0.6175
Epoch 5/10
37500/37500 [==============================] - 38s 1ms/step - loss: 0.8683 - acc: 0.6952 - val_loss: 1.0716 - val_acc: 0.6302
Epoch 6/10
37500/37500 [==============================] - 40s 1ms/step - loss: 0.8056 - acc: 0.7189 - val_loss: 1.1010 - val_acc: 0.6299
Epoch 7/10
37500/37500 [==============================] - 40s 1ms/step - loss: 0.7420 - acc: 0.7400 - val_loss: 1.1208 - val_a

__CNN Model with Dropout and Maxpool__
 - Will slide a 2x2 filter and take the max of the matrix
     - An x*x img will become a x/2 * x/2 img

In [13]:
cnn_model_dropout_maxpool = Sequential()
cnn_model_dropout_maxpool.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(32,32,3)))
cnn_model_dropout_maxpool.add(MaxPooling2D(pool_size=(2,2)))
cnn_model_dropout_maxpool.add(Dropout(0.25))
cnn_model_dropout_maxpool.add(Conv2D(32, kernel_size=3, activation='relu'))
cnn_model_dropout_maxpool.add(Dropout(0.25))

cnn_model_dropout_maxpool.add(Flatten())
cnn_model_dropout_maxpool.add(Dense(10, activation='softmax'))

In [15]:
cnn_model_dropout_maxpool.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
early_stopping_monitor = EarlyStopping(patience=3)
cnn_model_dropout_maxpool.fit(X_train, y_train,  epochs=10, verbose=1,validation_data=(X_test,y_test), callbacks=[early_stopping_monitor])

Train on 37500 samples, validate on 12500 samples
Epoch 1/10
37500/37500 [==============================] - 19s 511us/step - loss: 1.5773 - acc: 0.4336 - val_loss: 1.2821 - val_acc: 0.5513
Epoch 2/10
37500/37500 [==============================] - 19s 519us/step - loss: 1.2397 - acc: 0.5644 - val_loss: 1.1979 - val_acc: 0.5764
Epoch 3/10
37500/37500 [==============================] - 19s 499us/step - loss: 1.1116 - acc: 0.6091 - val_loss: 1.0900 - val_acc: 0.6175
Epoch 4/10
37500/37500 [==============================] - 18s 489us/step - loss: 1.0421 - acc: 0.6367 - val_loss: 1.0325 - val_acc: 0.6422
Epoch 5/10
37500/37500 [==============================] - 19s 499us/step - loss: 0.9909 - acc: 0.6530 - val_loss: 1.0056 - val_acc: 0.6526
Epoch 6/10
37500/37500 [==============================] - 19s 515us/step - loss: 0.9534 - acc: 0.6654 - val_loss: 0.9788 - val_acc: 0.6598
Epoch 7/10
37500/37500 [==============================] - 21s 573us/step - loss: 0.9149 - acc: 0.6825 - val_loss: 0.

__CNN Model with Dropout/MaxPool/BatchNormalization__
- Batch Normalization will normalize the values in the hidden layer

In [17]:
cnn_model_dropout_maxpool_batchnormalization = Sequential()
cnn_model_dropout_maxpool_batchnormalization.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(32,32,3)))
cnn_model_dropout_maxpool_batchnormalization.add(MaxPooling2D(pool_size=(2,2)))
cnn_model_dropout_maxpool_batchnormalization.add(Dropout(0.25))
cnn_model_dropout_maxpool_batchnormalization.add(BatchNormalization())
cnn_model_dropout_maxpool_batchnormalization.add(Conv2D(32, kernel_size=3, activation='relu'))
cnn_model_dropout_maxpool_batchnormalization.add(Dropout(0.25))

cnn_model_dropout_maxpool_batchnormalization.add(Flatten())
cnn_model_dropout_maxpool_batchnormalization.add(Dense(10, activation='softmax'))

In [18]:
cnn_model_dropout_maxpool_batchnormalization.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
early_stopping_monitor = EarlyStopping(patience=3)
cnn_model_dropout_maxpool_batchnormalization.fit(X_train, y_train,  epochs=10, verbose=1,validation_data=(X_test,y_test), callbacks=[early_stopping_monitor])

Train on 37500 samples, validate on 12500 samples
Epoch 1/10
37500/37500 [==============================] - 23s 621us/step - loss: 1.4841 - acc: 0.4699 - val_loss: 1.2769 - val_acc: 0.5562
Epoch 2/10
37500/37500 [==============================] - 24s 630us/step - loss: 1.1779 - acc: 0.5837 - val_loss: 1.2012 - val_acc: 0.5820
Epoch 3/10
37500/37500 [==============================] - 24s 652us/step - loss: 1.0769 - acc: 0.6256 - val_loss: 1.7890 - val_acc: 0.4668
Epoch 4/10
37500/37500 [==============================] - 24s 638us/step - loss: 1.0137 - acc: 0.6456 - val_loss: 1.7074 - val_acc: 0.4971
Epoch 5/10
37500/37500 [==============================] - 24s 643us/step - loss: 0.9595 - acc: 0.6662 - val_loss: 0.9927 - val_acc: 0.6533
Epoch 6/10
37500/37500 [==============================] - 24s 643us/step - loss: 0.9220 - acc: 0.6787 - val_loss: 1.0643 - val_acc: 0.6297
Epoch 7/10
37500/37500 [==============================] - 23s 617us/step - loss: 0.8996 - acc: 0.6862 - val_loss: 1.

__Data Augmentation__
 - The training data is very balanced, data augmentation might not make a big impact

In [ ]:
train_data,train_labels,test_data,test_labels=imgloader.getVisData()

In [ ]:
df =pd.DataFrame(train_labels,columns=['Label'])

In [ ]:
df['Label'].value_counts()

__Testing Accuracy__

In [21]:
score = cnn_model_dropout_maxpool_batchnormalization.evaluate(test_data, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0360902584075928
Test accuracy: 0.6406
